# M/S Vitaskär

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics
from scipy.stats import norm
from myst_nb import glue
#plt.style.use('presentation')
from IPython.display import display, Markdown, Latex
import sympy as sp
from d2e2f.visualization.visualize import plot_map, plot_trips

In [ ]:
%reload_kedro
df_stat = catalog.load('vitaskar.trip_statistics_clean')
#df_stat = catalog.load('uraniborg.trip_statistics')
df_stat['start_time'] = pd.to_datetime(df_stat['start_time'], utc=True)
df_stat['end_time'] = pd.to_datetime(df_stat['end_time'], utc=True);
#df_stat.describe()

In [ ]:
df_stat['item'] = 'raw'
df_clean = df_stat.copy()
df_clean['item'] = 'clean'
df_clean.dropna(subset=["Consumption ME1 (L)", "Consumption ME2 (L)"], inplace=True)



mask = df_clean['consumption'] > 0
df_clean = df_clean.loc[mask].copy()

#mask = df_clean['E'] > df_clean['E'].quantile(0.01) 
#df_clean = df_clean.loc[mask].copy()

In [ ]:
df_clean.head()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,5)
facegrid = sns.scatterplot(data=df_clean, x='start_time',y='consumption', ax=ax);
ax.set_ylabel('(Fuel consumption per trip) [l]');

fig = ax.get_figure()
fig_name = 'overview'
glue(fig_name, fig, display=False)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,5)
sns.histplot(df_clean, x='consumption', hue='trip_direction', ax=ax);

fig_name = 'fuel_hist'
glue(fig_name, fig, display=False)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,5)
sns.histplot(df_clean, x='sog', hue='trip_direction', ax=ax);

fig_name = 'sog_hist'
glue(fig_name, fig, display=False)

```{glue:figure} overview
:figwidth: 1000px
:name: "overview"

Fuel consumption over time
```

```{glue:figure} fuel_hist
:figwidth: 1000px
:name: "fuel_hist"

Histogram over fuel consumption in both directions.
```

```{glue:figure} sog_hist
:figwidth: 1000px
:name: "sog_hist"

Histogram over speed over ground in both directions.
```

In [ ]:
df_ = df_clean.set_index('start_time')
df_.sort_index(inplace=True)

tänd = '2022-02-01'
utbildning = '2022-07-13'

df_.loc[:tänd,'period'] = 'släckta skärmar'
df_.loc[tänd:utbildning,'period'] = 'tända skärmar'
df_.loc[utbildning:,'period'] = 'efter utbildning'

In [ ]:
df_expanding = df_.expanding().mean()

In [ ]:
def plot_periods(y='consumption', ylabel='Fuel consumption [liters]', fig_name = 'fuel'):
    
    fig,ax=plt.subplots()
    fig.set_size_inches(20,10)
    
    styles = ['k.','g.','b.']
    for style, (period, df_period) in zip(styles,df_.groupby(by='period', sort=False)):
        df_period.plot(y=y, label=period, ax=ax, style=style)
    df_expanding.loc[tänd:].plot(y=y, label=f'expanding mean', ax=ax, style='k-', lw=3)
    ax.set_ylabel(ylabel)
    ax.legend();
    
    
    fig = ax.get_figure()
    glue(fig_name, fig, display=False)
    
plot_periods()
#plot_periods(y='w', ylabel='Wind speed [m/s]', fig_name='wind')
plot_periods(y='trip_time', ylabel='trip time', fig_name='trip_times')


In [ ]:
trips = catalog.load("vitaskar.data_with_trip_columns")
harbours = catalog.load("params:vitaskar.harbours")
initial_speed_separator = catalog.load("params:initial_speed_separator")

In [ ]:
trip_nos = df_.groupby(by='period')['trip_no'].unique()
trip_nos

In [ ]:
mask = trips['trip_no'].isin(trip_nos['tända skärmar'])
plot_trips(df=trips.loc[mask], zoom_start=12)

In [ ]:
mask = trips['trip_no'].isin(trip_nos['efter utbildning'])
plot_trips(df=trips.loc[mask], zoom_start=12)

In [ ]:
df_.rolling(window='10D').mean().plot(y='consumption')

In [ ]:
df_.rolling(window='10D').sum().plot(y='consumption')

In [ ]:
df_.hist('consumption', bins=100)

In [ ]:
mask = df_.index <= '2022-07-13'
df_.loc[mask].mean()['consumption']

In [ ]:
mask = df_.index <= '2022-07-13'
df_.loc[mask].max()['consumption']

In [ ]:
mask = df_.index > '2022-07-13'
df_.loc[mask].mean()['consumption']

In [ ]:
mask = df_.index > '2022-07-13'
df_.loc[mask].max()['consumption']

In [ ]:
mask = trips.index <= utbildning
trips.loc[mask].describe()

In [ ]:
mask = trips.index > utbildning
trips.loc[mask].describe()

In [ ]:
mask = trips['trip_no'].isin(df_clean['trip_no'])
trips=trips.loc[mask]
trips.sort_index(inplace=True)
trips.head()
trips['time'] = (trips.index - trips.index[0]).total_seconds()
trips.head()

In [ ]:
trips['time'].diff().max()

In [ ]:
trips['time'].diff().plot()

In [ ]:
mask = trips['time'].diff() > 600
trips.loc[mask]['consumption'].max()

In [ ]:
index = trips.loc[mask].iloc[0].name
i = trips.index.get_loc(index)
trips.iloc[i-2:i+3]

In [ ]:
fig,ax=plt.subplots()
trips.iloc[i-10:i+10].plot(y='sog', style='.-', ax=ax)
ax.plot(trips.iloc[i].name, trips.iloc[i]['sog'],'go')
ax.plot(trips.iloc[i-1].name, trips.iloc[i-1]['sog'],'ro')

fig,ax=plt.subplots()
trips.iloc[i-10:i+10].plot(y='consumption', style='.-', ax=ax)
ax.plot(trips.iloc[i].name, trips.iloc[i]['consumption'],'go')
ax.plot(trips.iloc[i-1].name, trips.iloc[i-1]['consumption'],'ro')

plot_trips(df=trips.iloc[i-10:i+10], zoom_start=14)

In [ ]:
harbours

In [ ]:
harbour = harbours['Norgersund']
latitude = harbour[0]
longitude = harbour[1]

trips['distance to Norgersund'] = np.sqrt((trips.latitude - latitude)**2 + (trips.longitude - longitude)**2)


In [ ]:
trip = trips.groupby(by='trip_no').get_group(100)
plot_map(trip, zoom_start=13)

In [ ]:
trip.plot(y='distance to Norgersund')

In [ ]:
trip.head()

In [ ]:
distance_limit = 0.0004
mask = trips['distance to Norgersund'] < distance_limit
plot_trips(trips.loc[mask])

In [ ]:
np.roll(distance, -1)

In [ ]:
distance

In [ ]:
distance

In [ ]:
N = 5
distance = trips['distance to Norgersund']

M = np.concatenate([[np.roll(distance,-i)] for i in range(N,1,-1)])
average_previous_distance = M.mean(axis=0)

mask = ((distance <= distance_limit) & (average_previous_distance > distance_limit))
entering = trips.loc[mask]       

In [ ]:
def entering_or_leaving_harbour(harbour: list, trips :pd.DataFrame, entering=True, initial_speed_separator=0.1, distance_limit = 0.0004, N = 5):
                        
        
    latitude = harbour[0]
    longitude = harbour[1]
    
    distance = np.sqrt((trips.latitude - latitude)**2 + (trips.longitude - longitude)**2)
    sog = trips['sog']

    # Matrix with data shifted forward in time in the rows
    if N > 1:
        M = np.concatenate([[np.roll(distance,-i)] for i in range(N,1,-1)])
        average_previous_distance = M.mean(axis=0)
    else:
        average_previous_distance = np.roll(distance,-1)
        
    # If the ship is within the distance_limit of the harbour 
    # and the mean distance from the N previous time steps is larger, 
    # this is an entering harbour event.
    
    if entering:
        # Entering:
        mask = ((distance >= distance_limit) & (average_previous_distance < distance_limit))
    else:
        # Leaving:
        mask = ((distance <= distance_limit) & (average_previous_distance > distance_limit))
    
    events = trips.loc[mask]
    return events
        

    

In [ ]:
distance_limit = 0.0006
entering_harbour1 = entering_or_leaving_harbour(harbour=harbours['Norgersund'], trips=trips, entering=True, distance_limit=distance_limit, N=1)
leaving_harbour1 = entering_or_leaving_harbour(harbour=harbours['Norgersund'], trips=trips, entering=False, distance_limit=distance_limit, N=1)

In [ ]:
distance_limit = 0.0006
entering_harbour2 = entering_or_leaving_harbour(harbour=harbours['Hanö'], trips=trips, entering=True, distance_limit=distance_limit, N=1)
leaving_harbour2 = entering_or_leaving_harbour(harbour=harbours['Hanö'], trips=trips, entering=False, distance_limit=distance_limit, N=1)

In [ ]:
fig,ax=plt.subplots()
entering_harbour1.plot(y='sog', style='r^', label='entering Norgersund', ax=ax)
leaving_harbour1.plot(y='sog', style='go', label='leaving Norgersund', ax=ax)

entering_harbour2.plot(y='sog', style='rv', label='entering Hanö', ax=ax)
leaving_harbour2.plot(y='sog', style='gs', label='leaving Hanö', ax=ax)

trips.plot(y='sog', style='.-', label='sog', ax=ax, zorder=-10)
ax.set_xlim('2022-05-01','2022-05-02')
ax.legend()

In [ ]:
mask = trips['sog'] > 0.1

fig,ax=plt.subplots()
entering_harbour1.plot(y='sog', style='r^', label='entering Norgersund', ax=ax)
leaving_harbour1.plot(y='sog', style='go', label='leaving Norgersund', ax=ax)

entering_harbour2.plot(y='sog', style='rv', label='entering Hanö', ax=ax)
leaving_harbour2.plot(y='sog', style='gs', label='leaving Hanö', ax=ax)

trips.loc[mask].plot(y='sog', style='.-', label='sog', ax=ax, zorder=-10)
ax.set_xlim('2022-05-01','2022-05-02')
ax.legend()


In [ ]:
fig,ax=plt.subplots()
df_clean.plot(x='start_time', y='sog', style='go', label='trip start', ax=ax)
df_clean.plot(x='end_time', y='sog', style='r^', label='trip end', ax=ax)
trips.plot(y='sog', style='.-', label='sog', ax=ax, zorder=-10)
ax.set_xlim('2022-05-01','2022-05-02')
ax.legend()

In [ ]:
df_clean.head()

In [ ]:
mask.idxmax()

In [ ]:
interesting = ['sog','consumption']
df_compare = df_.groupby(by='period', sort=False).mean()[interesting]
table = df_compare.style.format(precision=2)
glue("periods", table)

```{glue:figure} fuel
:figwidth: 1000px
:name: "fuel"

Fuel consumption over time
```

```{glue:figure} wind
:figwidth: 1000px
:name: "wind"

Wind speed over time
```

```{glue:figure} periods
:name: "periods"

Mean values for each period.
```

It seems that the speed has decreased when going from Ven to Landskrona as seen in {numref}`sog_Ven-Landskrona` but not in the other direction as seen in {numref}`sog_Landskrona-Ven`.

```{glue:figure} sog_Ven-Landskrona
:figwidth: 1000px
:name: "sog_Ven-Landskrona"

Speed over ground: Ven-Landskrona
```

```{glue:figure} sog_Landskrona-Ven
:figwidth: 1000px
:name: "sog_Landskrona-Ven"

Speed over ground: Landskrona-Ven
```

The Aft thrust allocation factor is representing how much the aft azimuth thruster is contributing, where a value of 1.0 means that only the aft azimuth is working. It can be seen that this factor has changed for trips from Ven to Landskrona as seen in {numref}`thrust_factor_Ven-Landskrona`. For "släckta skärmar" the factor is most often 0.5 meaning a 50-50% split between the thrusters. This has changed towards having a larger contribution on the aft asimuth thruster. This change cannot be observered in the other direction as seen in {numref}`thrust_factor_Landskrona-Ven`.


```{glue:figure} thrust_factor_Ven-Landskrona
:figwidth: 1000px
:name: "thrust_factor_Ven-Landskrona"

Aft thrust allocation factor: Ven-Landskrona
```

```{glue:figure} thrust_factor_Landskrona-Ven
:figwidth: 1000px
:name: "thrust_factor_Landskrona-Ven"

Aft thrust allocation factor: Landskrona-Ven
```





In [ ]:
keys = ['sog',
        'consumption',
        'distance',
       ]


for key in keys:
    bins = np.linspace(df_[key].min(), df_[key].max(), 100)
    
    for direction, df_direction in df_.groupby(by='trip_direction', sort=False):
        grid = sns.displot(df_direction, x=key, kind='kde', hue='period', common_norm=False, bw_adjust=0.5);
        fig = grid.figure
        fig.suptitle(direction)
        
        fig_name = f'{key}_{direction}'
        glue(fig_name, fig, display=False)